In [1]:
import pandas as pd
merged_df = pd.read_excel('model 1 final data.xlsx')

In [2]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def prepare_sbert_data(data, review_column):
    data['임베딩'] = data[review_column].apply(lambda x: model.encode(x, convert_to_tensor=True))
    return data

sbert_data = prepare_sbert_data(merged_df, review_column="리뷰")


# 키워드 자동 추출 함수
def extract_keywords(data, column_name):
    """
    데이터의 특정 컬럼에서 고유 키워드를 추출합니다.
    - 중간에 구분자가 있는 경우(예: '・') 분리하여 고유값을 수집합니다.
    """
    unique_keywords = set()
    data[column_name].dropna().apply(lambda x: unique_keywords.update(x.split('・') if '・' in x else [x]))
    return list(unique_keywords)

location_keywords = extract_keywords(sbert_data, "요약주소")
category_keywords = extract_keywords(sbert_data, "가게분류") 
companion_keywords = extract_keywords(sbert_data, "1등 동료")
companion_keywords += extract_keywords(sbert_data, "2등 동료")
companion_keywords = list(set(companion_keywords)) 

# 사용자 입력에서 필터 추출 함수
def extract_filters(user_input, location_keywords, category_keywords, companion_keywords):
    location_filter = None
    category_filter = None
    companion_filter = None

    # 주소 키워드 탐색
    for keyword in location_keywords:
        if keyword in user_input:
            location_filter = keyword
            break
    
    # 가게분류 키워드 탐색
    for keyword in category_keywords:
        if keyword in user_input:
            category_filter = keyword
            break

    # 동료 키워드 탐색
    for keyword in companion_keywords:
        if keyword in user_input:
            companion_filter = keyword
            break
    
    return location_filter, category_filter, companion_filter



# 동료 데이터를 분리하는 함수
def split_companions(row, companion_column):
    return row[companion_column].split('・') if pd.notna(row[companion_column]) else []



# 필터링된 데이터 기반 추천 함수
def recommend_top_stores(user_input, sbert_data, top_n=3):

    user_embedding = model.encode(user_input, convert_to_tensor=True)


    location_filter, category_filter, companion_filter = extract_filters(user_input, location_keywords, category_keywords, companion_keywords)

    if location_filter and category_filter:
        filtered_data = sbert_data[
            (sbert_data['요약주소'].str.contains(location_filter, na=False)) & 
            (sbert_data['가게분류'].str.contains(category_filter, na=False))
        ]
    elif location_filter:
        filtered_data = sbert_data[
            sbert_data['요약주소'].str.contains(location_filter, na=False)
        ]
    elif category_filter: 
        filtered_data = sbert_data[
            sbert_data['가게분류'].str.contains(category_filter, na=False)
        ]
    else:  
        filtered_data = sbert_data

    # 각 리뷰와 사용자 입력 간의 유사도 계산
    store_scores = {}
    for _, row in filtered_data.iterrows():
        # 기본 유사도 계산
        similarity = util.pytorch_cos_sim(user_embedding, row['임베딩']).item()

        if companion_filter:
            # 1등 동료 분리 및 가중치 반영
            companions_1st = split_companions(row, '1등 동료')
            for companion in companions_1st:
                if companion_filter in companion:
                    similarity += row['1등 비율'] * 0.1  # 1등 비율을 가중치로 추가
            
            # 2등 동료 분리 및 가중치 반영
            companions_2nd = split_companions(row, '2등 동료')
            for companion in companions_2nd:
                if companion_filter in companion:
                    similarity += row['2등 비율'] * 0.1  # 2등 비율을 가중치로 추가
        
        store = row['가게명']
        # 가게별 최고 유사도를 갱신
        store_scores[store] = max(store_scores.get(store, 0), similarity)

    # 유사도를 기준으로 상위 N개의 가게 추천
    top_stores = sorted(store_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    # 결과를 데이터프레임으로 변환
    recommended_stores = pd.DataFrame(top_stores, columns=['가게명', '유사도'])
    
    # 추가 정보 병합
    recommended_stores = pd.merge(
        recommended_stores, 
        sbert_data[['가게명', '주소', '가게분류', '1등 웨이팅', '2등 웨이팅', '1등 동료', '2등 동료', '1등 비율', '리뷰']],
        on='가게명',
        how='left'
    )
    return recommended_stores

# 유사도 평가 함수 추가
def evaluate_recommendations(user_input, recommended_stores, review_column='요약리뷰'):
    """
    추천된 가게와 사용자 입력 간의 리뷰 유사도 및 최종 유사도를 계산합니다.
    """
    user_embedding = model.encode(user_input, convert_to_tensor=True)

    # 리뷰 유사도 계산
    recommended_stores['리뷰 유사도'] = recommended_stores[review_column].apply(
        lambda x: util.pytorch_cos_sim(user_embedding, model.encode(x, convert_to_tensor=True)).item()
    )

    # 최종 유사도(기존 계산된 유사도와 비교 가능)
    avg_review_similarity = recommended_stores['리뷰 유사도'].mean()
    avg_final_similarity = recommended_stores['유사도'].mean()

    print(f"평균 리뷰 유사도: {avg_review_similarity:.4f}")
    print(f"평균 최종 유사도: {avg_final_similarity:.4f}")
    print("\n추천된 가게별 유사도:\n")

    # 추천된 가게 정보 출력
    for idx, row in recommended_stores.iterrows():
        print(f"가게명: {row['가게명']}({row['가게분류']})")
        print(f"주소: {row['주소']}")
        print(f"리뷰 유사도: {row['리뷰 유사도']:.4f}")
        print(f"최종 유사도: {row['유사도']:.4f}")
        print(f"{round(row['1등 비율'] * 100, 1)}%의 사람들이 {row['1등 동료']}와 함께 왔습니다.")
        print(f"많은 사람들이 {row['1등 웨이팅']}분의 대기시간을 가졌습니다.\n")

# 사용자 입력 예시
user_input = input("장소, 동료, 분위기, 음식 등 원하는 조건을 입력하세요: ")

# 상위 3개 가게 추천
recommendations = recommend_top_stores(user_input, sbert_data, top_n=3)

# 유사도 평가 및 출력
evaluate_recommendations(user_input, recommendations, review_column='리뷰')


c:\Users\Moai\anaconda3\envs\new_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


평균 리뷰 유사도: 0.7352
평균 최종 유사도: 0.7352

추천된 가게별 유사도:

가게명: 굴과찜사랑(한식)
주소: 서울 성동구 행당로17길 23
리뷰 유사도: 0.7394
최종 유사도: 0.7394
42.5%의 사람들이 친구와 함께 왔습니다.
많은 사람들이 0.0분의 대기시간을 가졌습니다.

가게명: 갈매기의꿈(한식)
주소: 서울 성동구 무학로2나길 1 1층 갈매기의꿈
리뷰 유사도: 0.7335
최종 유사도: 0.7335
35.2%의 사람들이 연인・배우자와 함께 왔습니다.
많은 사람들이 0.0분의 대기시간을 가졌습니다.

가게명: 갱생(한식)
주소: 서울 성동구 마조로7길 5 지하1층 갱생
리뷰 유사도: 0.7328
최종 유사도: 0.7328
53.2%의 사람들이 친구와 함께 왔습니다.
많은 사람들이 0.0분의 대기시간을 가졌습니다.

